In [ ]:
import pandas as pd
import numpy as np

In [98]:
data = pd.read_csv("Data/Neubauwohnungen_Zürich.csv")

aggregation_nach_Kreis = (
    data
    .groupby(["KreisLang", "FuenfjahresPeriode_noDM"])
    .agg(
        AnzRentner=("AnzRentner", "sum"),
        AnzKinder=("AnzKinder", "sum"),
        Wohnfläche=("Wohnflaeche", "sum"),
        Anz_Zimmer=("AnzZimmerLevel2Cd_noDM", "sum"),
        AnzWohnungen=("AnzWhgStat", "sum"),
        Bevölkerungsmenge=("AnzBestWir", "sum"),
        Mietwohnungen=("MietwohnungSort", lambda x: (x == 1).sum()))
    .reset_index()
    .rename(columns={"KreisLang": "Kreis", "FuenfjahresPeriode_noDM": "Periode"})

)

aggregation_nach_Kreis

,Kreis,Periode,AnzRentner,AnzKinder,Wohnfläche,Anz_Zimmer,AnzWohnungen,Bevölkerungsmenge,Mietwohnungen
0,Kreis 10,2015–2019,138,521,81799,527,878,2051,120
1,Kreis 10,2020–2024,53,207,39621,425,446,963,88
2,Kreis 11,2015–2019,282,645,152628,1127,1694,3545,250
3,Kreis 11,2020–2024,309,989,193924,974,2389,5056,247
4,Kreis 12,2015–2019,402,1003,157775,516,1779,4327,154
5,Kreis 12,2020–2024,216,197,45411,402,582,1145,107
6,Kreis 2,2015–2019,218,921,133098,714,1412,3506,163
7,Kreis 2,2020–2024,178,824,137466,645,1623,3631,165
8,Kreis 3,2015–2019,192,425,62254,499,740,1722,129
9,Kreis 3,2020–2024,45,207,59908,414,783,1495,122


In [97]:
data = pd.read_csv("Data/Neubauwohnungen_Zürich.csv")

aggregation_nach_quartier = (
    data
    .groupby(["QuarLang", "FuenfjahresPeriode_noDM"])
    .agg(
        AnzRentner=("AnzRentner", "sum"),
        AnzKinder=("AnzKinder", "sum"),
        Wohnfläche=("Wohnflaeche", "sum"),
        Anz_Zimmer=("AnzZimmerLevel2Cd_noDM", "sum"),
        AnzWohnungen=("AnzWhgStat", "sum"),
        Bevölkerungsmenge=("AnzBestWir", "sum"),
        Mietwohnungen=("MietwohnungSort", lambda x: (x == 1).sum()))
    .reset_index()
    .rename(columns={"KreisLang": "Kreis", "FuenfjahresPeriode_noDM": "Periode"})

)

aggregation_nach_quartier

,QuarLang,Periode,AnzRentner,AnzKinder,Wohnfläche,Anz_Zimmer,AnzWohnungen,Bevölkerungsmenge,Mietwohnungen
0,Affoltern,2015–2019,85,190,40739,371,427,966,84
1,Affoltern,2020–2024,88,310,46245,252,549,1283,59
2,Albisrieden,2015–2019,124,845,136869,364,1402,3389,83
3,Albisrieden,2020–2024,44,251,60231,315,760,1506,73
4,Alt-Wiedikon,2015–2019,16,118,13083,199,130,368,36
5,Alt-Wiedikon,2020–2024,18,125,35737,213,441,857,51
6,Altstetten,2015–2019,184,571,143282,416,1765,3653,108
7,Altstetten,2020–2024,179,672,122391,477,1495,3244,111
8,Enge,2015–2019,21,105,19339,192,190,434,38
9,Enge,2020–2024,30,47,10335,133,120,259,39
